##### 官方评测算子
在配置文件中或直接给Runner传入val_evaluator或test_evaluator参数，单个评测指标传入字典，多个评测指标传入字典组成的列表

In [ ]:
val_evaluator = dict(type='Accuracy', top_k=(1, 5))  # 使用分类正确率评测指标
test_evaluator = [
    # 目标检测指标
    dict(
        type='CocoMetric',
        metric=['bbox', 'segm'],
        ann_file='annotations/instances_val2017.json',
    ),
    # 全景分割指标
    dict(
        type='CocoPanopticMetric',
        ann_file='annotations/panoptic_val2017.json',
        seg_prefix='annotations/panoptic_val2017',
    )
]

##### 自定义评测指标
1. 继承BaseMetrics，实现process
2. 实现compute_metrics
3. 可选default_prefix参数

default_prefix参数为评价指标的前缀，是评测算子的类属性，也可在配置文件中通过prefix参数指定

In [ ]:
from mmengine.registry import METRICS
from mmengine.evaluator import BaseMetric
import numpy as np
from typing import Sequence

@METRICS.register_module()
class SimpleAccuracy(BaseMetric):
    default_prefix = 'Acc'
    def process(self,data_batch:Sequence[dict],data_samples:Sequence[dict]):
        #data_batch是来自于dataloader的数据，data_samples是来自于模型的输出
        result={
            'pred':data_samples['pred_label'],#bs*1
            'gt':data_samples['data_sample']['gt_label']
        }
        self.results.append(result)

    def compute_metrics(self,results):
        preds=np.concatenate([result['pred'] for result in results])
        gts=np.concatenate([result['gt'] for result in results])

        acc=(preds==gts).sum()/len(gts)
        return {'accuracy':acc}